# Image Downloader for Google Images

In [120]:
from google_images_download import google_images_download
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import urllib.request
import hashlib
import os
from PIL import Image
import io
from selenium.webdriver.common.keys import Keys

In [121]:
dataset=pd.read_csv("Task2_Dataset.csv")
spname=[]
for x in range(0,dataset.shape[0]):
    if dataset["species_name"].iloc[x] == '-1':
        continue
    elif x not in spname:
        spname.append(dataset["species_name"].iloc[x])
spname=list(set(spname))
print(len(spname))
spname[0]

309


'cercion calamorum'

In [122]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [123]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [126]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=15):
    target_folder = os.path.join(target_path,search_term.lower())

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=8)
        
    for elem in res:
        persist_image(target_folder,elem)

In [128]:
for x in range(0,spname.shape[0]):
    driverpath="/home/sic_mundus/Downloads/chromedriver_linux64/chromedriver"
    search_and_download(spname[x],driverpath,"/home/sic_mundus/Documents/dragonfly-species-identification/Img1",30)

Found: 200 search results. Extracting links from 0:200
Found: 30 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRjz1c6Nxzt5y9oZ3pDgbc_9ehmaO246ws3KPeKMEHDncj_Fccp&usqp=CAU - as /home/sic_mundus/Documents/dragonfly-species-identification/Img1/merogomphus/2b98f1de56.jpg
SUCCESS - saved https://live.staticflickr.com/2907/14452379049_85ee3ce235_n.jpg - as /home/sic_mundus/Documents/dragonfly-species-identification/Img1/merogomphus/0eb238de76.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQeo2ce8TJ3C1gaF2UFqV4sAI2qFe-YqcL65xr1UBC6WqJZ3jX0&usqp=CAU - as /home/sic_mundus/Documents/dragonfly-species-identification/Img1/merogomphus/4cdd384700.jpg
SUCCESS - saved https://www.researchgate.net/profile/Oleg_Kosterin/publication/324519960/figure/fig46/AS:615298088587285@1523709904653/A-teneral-female-of-Merogomphus-parvus-photographed-at-Thma-Bang-River-bank-on.png - as /home/sic_mundus/Documents/dragonfly-species-identificatio